ESLint problems

This report shows the ESLint problems grouped by rule and then by repo.

In [ ]:
import plotly.express as px
import pandas as pd
from code_data_science import data_table as dt, data_grid as moderne_data_grid
import code_data_science.palette as palette

df = dt.read_csv("../samples/eslint_problems.csv")
df["problems"] = 1

# Exit early if there are no rows and render a plot with a message
if len(df) == 0:
    fig = px.treemap(names=[], parents=[])
    fig.add_annotation(
        x=0.5, y=0.5, text="No rows of data found", showarrow=False, font=dict(size=20)
    )
else:
    # concat origin:path:branch into repository
    df["repository"] = (
        df["repositoryOrigin"]
        + ":"
        + df["repositoryPath"]
        + ":"
        + df["repositoryBranch"]
    )
    df["fileType"] = df["sourcePath"].str.split(".").str[-1]
    df_file_grouped = df.groupby(
        by=["ruleId", "repository", "fileType"],
        as_index=False,
    ).sum(numeric_only=True)

    path = ["ruleId", "repository"]

    # number of rows in the dataframe
    total_problems = len(df["sourcePath"])

    # number of rows with parse failures
    total_fatal = len(df[df["fatal"] == True])

    measurement_field = "problems"

    # Set the color palette
    colors = palette.qualitative()

    fig = px.treemap(
        df_file_grouped,
        path=path,
        values=measurement_field,
        color_discrete_sequence=colors,
    )

    fig.update_layout(
        margin=dict(t=28, l=0, r=0, b=0),
        annotations=[
            dict(
                x=0.01,
                y=1.055,
                showarrow=False,
                borderwidth=1,
                text="<b>More info</b>",
                hovertext=(
                    f"""
            <b>Total problems:</b> {"{:,}".format(total_problems)} </br> </br>
            """
                ),
                xref="paper",
                yref="paper",
            )
        ],
    )

    fig.update_traces(
        marker=dict(cornerradius=3),
    )

    fig.data[0].textinfo = "label+text+value"

In [ ]:
# Output the visualization
fig.show(render="plotly_mimetype")